In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'real-or-fake-fake-jobposting-prediction:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F533871%2F976879%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240313%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240313T190432Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db8c076ba29b87965728e62427df498b6be578fd6f58a33f42ec3aedeec3e117c945e28f9403296e14f2a83f1a1cb35282faa4fd9c171c9931c5f6e518cd9ebc8e7b03f1419aad6d4e1091ac478df2abbc7bee654a3f7e0afb53f4513ed23e91c111199491ac8e3355abdafe6277f1f916dec45e4d37e2f2b0ee90bf9c9cf429946b0781c63148156bf29a31d0a633ae37eff923f81836ed4f8ed0abdf47633fcc2c7785317771e8cacc1a7c04437d580297e1aa2cb1ce4b5b0fa4c96944c45a906023b478ab45a977d673fdfae16232b14693b49580b6d36d88d46dbbcac653a8f7c5c251822d6b3fd9b27a1e3e139ee1c060c7bafee675958d05a4ac3f57d42'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Load the dataset
train_df = pd.read_csv('/kaggle/input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv')

In [ ]:
#Explore first five rows
train_df.head()

In [ ]:
#Taking useful columns only
df = train_df[['description','fraudulent']]

In [ ]:
#Checking new dataframe
df.head()

# **Handle null values**

In [ ]:
#handle null values
df.isna().sum()

In [ ]:
#drop null values
df.dropna(inplace=True,axis=0)

In [ ]:
df.shape

# **Distribution of the categories**

In [ ]:
df['fraudulent'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.countplot(data = df,x='fraudulent')
plt.title('Category distribution')
plt.show()

# **Text preprocessing**

In [ ]:
import re
def process_text(text):
    news = re.sub(r'[^a-zA-Z\s]','',text)
    lo_news = news.lower()
    return lo_news


In [ ]:
df['description'] = df['description'].apply(process_text)

# **vectorization,model building**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.layers import LSTM,GRU,Bidirectional,Dense,Embedding
from tensorflow.keras import Sequential

In [ ]:
# passing object
tk = Tokenizer()

In [ ]:
#fit the text into the tokenizer
tk.fit_on_texts(df['description'])

In [ ]:
#integer encoding
seq = tk.texts_to_sequences(df['description'])

In [ ]:
#padded the vector to equalize the dimenstion
vec = pad_sequences(seq,padding='post',maxlen=50)

In [ ]:
# split features and target variable
import numpy as np
x = np.array(vec)
y = np.array(df['fraudulent'])

In [ ]:
#split training and tesing set

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=32)

# ******Model**

In [ ]:
# Building model using sequential API

model = Sequential()
model.add(Embedding(input_dim=len(tk.word_index)+1,output_dim=100,input_length=50))
model.add(Bidirectional(LSTM(units=100)))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,y_train,epochs=2,batch_size=32,validation_data=(x_test,y_test))

In [ ]:
pd.DataFrame(history.history).plot()

In [ ]:
#Evaluate the result
model.evaluate(x_test,y_test)

In [ ]:
#prediction

y_pred_prob = model.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# **Metrics**

In [ ]:
#Measure confustion matrix
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,f1_score,accuracy_score,precision_score
cnf = confusion_matrix(y_test,y_pred)

In [ ]:
ax= ConfusionMatrixDisplay(cnf,display_labels=['Not fake','fake']).plot()